lack of no. of points info

In [7]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [8]:
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [9]:
pip install python-geohash

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
import time
import geohash
import logging
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [11]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [12]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
pe_tj_table = f"{schema_name['cda']}.trajectory_uplevelled"

In [16]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection setup
output_schema_name = 'od_matrix20'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

In [17]:
# Function to process data for a single day
def process_day(event_date, country_code, sql_engine):
    try:
        # Read data from the SQL table
        pe_tj_df = sql_engine.read_sql(
            f"""
            SELECT 
                cuebiq_id,
                start_lat,
                start_lng,
                end_lat,
                end_lng,
                duration_minutes,
                length_meters,
                number_of_points
            FROM cuebiq.paas_cda_pe_v3.trajectory_uplevelled
            WHERE 
                event_date = {event_date}
                AND end_country = '{country_code}' 
                AND start_country = '{country_code}' 
            """
        )
        logging.info(f"Executing SQL query for date {event_date}")

        # Encode geohashes
        pe_tj_df['start_geohash3'] = pe_tj_df.apply(
            lambda x: geohash.encode(x['start_lat'], x['start_lng'], precision=3), axis=1)
        pe_tj_df['end_geohash3'] = pe_tj_df.apply(
            lambda x: geohash.encode(x['end_lat'], x['end_lng'], precision=3), axis=1)

        # Aggregate data
        aggregated_df3 = pe_tj_df.groupby(['start_geohash3', 'end_geohash3']).agg({
            'cuebiq_id': 'count',
            'duration_minutes': ['mean', 'median', 'std'],
            'length_meters': ['mean', 'median', 'std'],
            'number_of_points': ['mean', 'median', 'std']
        }).reset_index()
        aggregated_df3.columns = ['start_geohash3', 'end_geohash3', 'trip_count', 
                                  'm_duration_min', 'mdn_duration_min', 'sd_duration_min', 
                                  'm_length_m', 'mdn_length_m', 'sd_length_m',
                                  'm_points_no', 'mdn_points_no', 'sd_points_no']

        # Filter aggregated data and reorder columns
        filtered_df3= aggregated_df3.loc[aggregated_df3['trip_count'] > 9]
        filtered_df3 = filtered_df3[['start_geohash3', 'end_geohash3', 'trip_count', 
                                     'm_duration_min', 'mdn_duration_min', 'sd_duration_min', 
                                     'm_length_m', 'mdn_length_m', 'sd_length_m',
                                     'm_points_no', 'mdn_points_no', 'sd_points_no']]
        return filtered_df3

    except Exception as e:
        logging.error(f"Error processing data for date {event_date}: {e}")
        return pd.DataFrame()

# Function to insert data with retry mechanism
def insert_data_with_retry(df, table_name, con, retries=3, delay=5):
    for attempt in range(retries):
        try:
            df.to_sql(
                table_name, 
                con, 
                index=False, 
                if_exists="append", 
                method="multi"
            )
            logging.info(f"Inserted data into table {table_name}")
            break
        except Exception as e:
            logging.error(f"Attempt {attempt+1} failed with error: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                logging.error(f"Failed to insert data into table {table_name} after {retries} attempts")

# Main processing loop
def process_date_range(start_date, end_date, country_code, sql_engine):
    start_time = time.time()  # Record start time before processing loop
        
    current_date = start_date
    while current_date <= end_date:
        event_date = current_date.strftime('%Y%m%d')
        
        filtered_df3 = process_day(event_date, country_code, sql_engine)

        # Insert data into the database
        if not filtered_df3.empty:
            # table_name = f"{output_schema_name}.{country_code}_agg3"
            table_name = f"od_{country_code.lower()}_{event_date}_agg3_10"
            insert_data_with_retry(filtered_df3, table_name, con)
        else:
            logging.info(f"No data to insert for date {event_date}")

        # Move to the next day
        current_date += timedelta(days=1)
    end_time = time.time()  # Record end time after processing loop
    total_time = end_time - start_time
    logging.info(f"Total processing time: {total_time:.2f} seconds")

In [18]:
# Example usage:
process_date_range(datetime(2020, 1, 1), datetime(2020, 12, 31), 'ID', sql_engine)

2024-06-17 18:07:42,585 - INFO - Executing SQL query for date 20190101
2024-06-17 18:07:47,787 - INFO - Inserted data into table od_id_20190101_agg3_10
2024-06-17 18:08:01,937 - INFO - Executing SQL query for date 20190102
2024-06-17 18:08:06,552 - INFO - Inserted data into table od_id_20190102_agg3_10
2024-06-17 18:08:20,590 - INFO - Executing SQL query for date 20190103
2024-06-17 18:08:25,408 - INFO - Inserted data into table od_id_20190103_agg3_10
2024-06-17 18:08:39,944 - INFO - Executing SQL query for date 20190104
2024-06-17 18:08:44,720 - INFO - Inserted data into table od_id_20190104_agg3_10
2024-06-17 18:08:58,618 - INFO - Executing SQL query for date 20190105
2024-06-17 18:09:03,398 - INFO - Inserted data into table od_id_20190105_agg3_10
2024-06-17 18:09:17,087 - INFO - Executing SQL query for date 20190106
2024-06-17 18:09:21,517 - INFO - Inserted data into table od_id_20190106_agg3_10
2024-06-17 18:09:37,342 - INFO - Executing SQL query for date 20190107
2024-06-17 18:09: